<a href="https://colab.research.google.com/github/DBCooper200/project_dl/blob/main/resnet_modified_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 May 23 17:30 kaggle.json


In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d himanshuagarwal1998/glaucomadataset

100% 2.53G/2.54G [00:26<00:00, 196MB/s]
100% 2.54G/2.54G [00:26<00:00, 103MB/s]


In [7]:
!unzip -q /content/glaucomadataset.zip

In [8]:
!mkdir 'Data_set'

In [9]:
import shutil, sys
shutil.move("/content/Non Glaucoma", "/content/Data_set")

'/content/Data_set/Non Glaucoma'

In [10]:
shutil.move("/content/Glaucoma", "/content/Data_set")

'/content/Data_set/Glaucoma'

In [11]:
'''FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
'''

'FAST_RUN = False\nIMAGE_WIDTH=128\nIMAGE_HEIGHT=128\nIMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)\nIMAGE_CHANNELS=3\n'

In [12]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing import image
import os
import cv2
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')

In [13]:
path = ['/content/Data_set/Non Glaucoma', '/content/Data_set/Glaucoma']
images = []
labels = []
for n,i in enumerate(path):
    for j in tqdm(os.listdir(i)):
        img_path = os.path.join(i,j)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64,64))
        images.append(img)
        labels.append(n)
images = np.array(images)/255
labels = np.array(labels)

100%|██████████| 511/511 [00:10<00:00, 47.53it/s]


In [14]:
for n in enumerate(path):
  print(n)

#the labels assigned are 
# 0 for non glaucomic images
# 1 for glaucomic images

(0, '/content/Data_set/Non Glaucoma')
(1, '/content/Data_set/Glaucoma')


In [15]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, labels = shuffle(images, labels, random_state=32)
x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.15, random_state=44)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=40)

In [16]:

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                            fill_mode='constant', cval=0.)
train_gen = datagen.flow(x_train, y_train, batch_size=32)
#Using TensorFlow backend.

#for test set
test_gen = datagen.flow(x_test, y_test, batch_size = 32)

In [17]:
HEIGHT = 64
WIDTH = 64
BATCH_SIZE = 32

In [18]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
'''
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7
'''
from sklearn.metrics import classification_report
import plotly.express as px
import pandas as pd

In [19]:
IMAGE_SIZE = (64,64)

In [20]:
resnet =ResNet50(weights= None, include_top=False, input_shape= (HEIGHT,WIDTH,3))
#vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#xception = Xception(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#resnet101 = ResNet101(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#inception = InceptionV3(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#mobilenet = MobileNetV2(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#efficientnet = EfficientNetB7(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

In [34]:
from keras.optimizers import SGD, Adam
from keras.layers import Dropout,Dense, GlobalAveragePooling2D
x = x = layers.Flatten()(resnet.output)
predictions = Dense(1,activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)
adam=keras.optimizers.Adam(
   learning_rate = 0.0001
)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
tb_callback = tensorflow.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)
model.fit_generator(train_gen, epochs=100, steps_per_epoch=1,verbose=1, validation_data=(x_valid,y_valid),callbacks=[tb_callback])

Epoch 1/100
1/1 [==============================] - 14s 14s/step - loss: 1.6152 - accuracy: 0.4062 - val_loss: 2.0280 - val_accuracy: 0.5390
Epoch 2/100
1/1 [==============================] - 6s 6s/step - loss: 0.7517 - accuracy: 0.4375 - val_loss: 3.1303 - val_accuracy: 0.5390
Epoch 3/100
1/1 [==============================] - 6s 6s/step - loss: 0.5473 - accuracy: 0.4688 - val_loss: 3.6495 - val_accuracy: 0.5390
Epoch 4/100
1/1 [==============================] - 7s 7s/step - loss: 0.4948 - accuracy: 0.6250 - val_loss: 3.2342 - val_accuracy: 0.5390
Epoch 5/100
1/1 [==============================] - 7s 7s/step - loss: 0.5036 - accuracy: 0.6875 - val_loss: 2.7868 - val_accuracy: 0.5390
Epoch 6/100
1/1 [==============================] - 6s 6s/step - loss: 0.3369 - accuracy: 0.4375 - val_loss: 2.5816 - val_accuracy: 0.5390
Epoch 7/100
1/1 [==============================] - 7s 7s/step - loss: 0.3671 - accuracy: 0.3750 - val_loss: 2.5946 - val_accuracy: 0.5390
Epoch 8/100
1/1 [===============

In [ ]:
!tensorboard dev upload --logdir logs 

2023-05-21 18:09:31.798056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   YNT-YRJ-FCC


Upload started and will continue reading any new data as it's added to the logdir.

To sto

In [36]:
model.save('path_to_my_model.h5')

new_model = keras.models.load_model('path_to_my_model.h5')

In [37]:
pred = new_model.evaluate(x_test, y_test)

5/5 [==============================] - 2s 146ms/step - loss: 1.4298 - accuracy: 0.5496
